In [42]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
pd.set_option('display.max_columns', 400)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os
from pathlib import Path
os.chdir('..')
DATA = Path('_data/genomics-data')

In [62]:
from lightgbm import LGBMClassifier
from scipy.io import loadmat
from SVMExperiments.utils import load_mat

def classify_feature_rank(dataset, feature_set, treatment, 
                          pval_pos_threshold, num_bags, num_runs,
                          num_top_features, test_all_features=False):
    DataCV_dir      = DATA/'DataCV'
    Bags_dir        = DATA/'Bags'
    FeatRanking_dir = DATA/'FeatRanking'
    Log_dir         = DATA/'LogsClassifFeatureRank'
    Classifiers = [LGBMClassifier]
    run = 1
    
#     X = load_mat(DataCV_dir/f'{feature_set}/{dataset}_{treatment}({pval_pos_threshold}).trn.mat')
#     T = load_mat(DataCV_dir/f'{feature_set}/{dataset}_{treatment}({pval_pos_threshold}).tst.mat')
    
    FeatRank = pd.read_csv(FeatRanking_dir/f'{feature_set}/{dataset}_{treatment}({pval_pos_threshold}).csv', sep=';')
    Bags     = loadmat(Bags_dir/f'{dataset}({pval_pos_threshold})_Bags{run}.mat')
    
    FeatRankRows = FeatRank[FeatRank.Run == run]
    for idx, FeatRankRow in FeatRankRows.iterrows():
        for classifier in Classifiers:
            for num_top_feats in num_top_features:
                features = list(FeatRankRow[2:2+num_top_feats])
                print(features)
    
    return FeatRank, Bags
#     return X, T

In [33]:
%%time
X, T = classify_feature_rank('Erk', 'Dc', 'BCR', 0.01, 300, 5, [2,4,6,8,10,12], True)

CPU times: user 12.8 s, sys: 300 ms, total: 13.1 s
Wall time: 13.2 s


In [64]:
FeatRank, Bags = classify_feature_rank('Erk', 'Dc', 'BCR', 0.01, 300, 5, [2,4,6,8,10,12], True)

[3, 5]
[3, 5, 4, 10]
[3, 5, 4, 10, 2, 21]
[3, 5, 4, 10, 2, 21, 20, 30]
[3, 5, 4, 10, 2, 21, 20, 30, 31, 42]
[3, 5, 4, 10, 2, 21, 20, 30, 31, 42, 19, 33]
[4, 3]
[4, 3, 17, 8]
[4, 3, 17, 8, 10, 2]
[4, 3, 17, 8, 10, 2, 39, 5]
[4, 3, 17, 8, 10, 2, 39, 5, 24, 18]
[4, 3, 17, 8, 10, 2, 39, 5, 24, 18, 21, 22]
[3, 4]
[3, 4, 5, 10]
[3, 4, 5, 10, 2, 21]
[3, 4, 5, 10, 2, 21, 20, 6]
[3, 4, 5, 10, 2, 21, 20, 6, 30, 31]
[3, 4, 5, 10, 2, 21, 20, 6, 30, 31, 17, 42]
[4, 17]
[4, 17, 8, 3]
[4, 17, 8, 3, 10, 5]
[4, 17, 8, 3, 10, 5, 2, 22]
[4, 17, 8, 3, 10, 5, 2, 22, 39, 21]
[4, 17, 8, 3, 10, 5, 2, 22, 39, 21, 37, 18]


In [65]:
FeatRank.head()

,Run,Method,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45
0,1,Entropy,3,5,4,10,2,21,20,30,31,42,19,33,32,17,6,14,40,8,29,39,24,41,38,18,23,22,45,37,25,36,26,11,7,15,12,9,34,13,35,16,27,1,43,28,44
1,1,Ttest,4,3,17,8,10,2,39,5,24,18,21,22,37,29,6,31,30,16,20,15,27,34,40,33,9,12,38,23,19,35,11,25,42,32,7,36,1,26,14,41,45,13,44,43,28
2,1,Brattacharyya,3,4,5,10,2,21,20,6,30,31,17,42,19,8,33,39,32,29,24,22,40,37,18,14,38,23,36,41,15,45,25,11,26,7,9,12,16,34,1,35,13,27,43,28,44
3,1,Wilcoxon,4,17,8,3,10,5,2,22,39,21,37,18,24,6,29,20,12,45,15,30,26,16,31,1,19,34,23,38,9,33,27,11,25,32,40,7,28,36,35,41,13,42,14,44,43
4,2,Entropy,3,5,4,10,2,21,20,30,31,42,19,33,32,17,6,14,8,40,29,39,24,41,38,18,23,22,37,45,25,26,36,11,15,7,12,9,34,13,35,16,27,1,43,28,44


In [31]:
X.head()

,CauseGene,EffectGene,Replicate,Treatment,Pvalue,X0min_neg_effect-X0min_neg_cause,X15min_effect-X0min_neg_cause,X15min_effect-X15min_cause,X30min_effect-X0min_neg_cause,X30min_effect-X15min_cause,...,X90min_effect-X90min_cause,X120min_effect-X0min_neg_cause,X120min_effect-X15min_cause,X120min_effect-X30min_cause,X120min_effect-X90min_cause,X120min_effect-X120min_cause,X180min_effect-X0min_neg_cause,X180min_effect-X15min_cause,X180min_effect-X30min_cause,Target
0,CTNNB1,CSRP2,1,BCR,0.000,0.258286,-0.040952,1.646857,-0.074884,-0.536817,...,1.330743,0.084049,0.081471,-0.193222,0.075790,0.302063,0.032332,0.404781,-0.261072,1
1,CTNNB1,CSRP2,2,BCR,0.000,1.855353,0.386690,1.853610,0.402003,0.632198,...,0.537406,0.593482,0.584197,0.395299,-0.183978,1.384277,0.099596,-0.117455,-0.089068,1
2,CTNNB1,CSRP2,3,BCR,0.000,2.424238,-0.189336,0.336505,-0.069250,0.093951,...,1.696790,0.199627,-0.119745,-0.179242,0.201025,0.470845,-0.112967,-0.028793,0.049267,1
3,CTNNB1,DDIT4,1,BCR,0.002,0.292199,0.105321,0.510985,0.122684,0.378946,...,1.939397,0.184178,0.177948,0.216048,0.091588,0.321739,-0.018615,0.077717,0.074726,1
4,CTNNB1,DDIT4,2,BCR,0.002,0.397289,0.047185,0.708738,-0.061738,0.156178,...,0.324054,-0.010790,-0.177104,-0.100049,0.105843,0.189795,-0.008936,-0.184815,-0.114929,1
